# Rectangle Algebra

In [1]:
import qualreas as qr
import os
import numpy as np

path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

## Load the 2D Point Algebra

The 2-dimensional point algebra is based on the eight compass directions, N, S, E, W, NW, NE, SW, SE, and EQ (equals), and it is derived in the second half of the Jupyter notebook, "derive_point_algebras.ipynb".

In [2]:
ptalg2d = qr.Algebra(os.path.join(path, "Algebras/2D_Point_Algebra.json"))

In [3]:
ptalg2d.summary()

  Algebra Name: 2D_Point_Algebra
   Description: Autogenerated 2-dimensional point algebra
 Equality Rels: EQ
     Relations:
            NAME (SYMBOL)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE   DOMAIN        RANGE
              South (  S)               North (  N)    False      False       True       2DPt          2DPt
             Equals ( EQ)              Equals ( EQ)     True       True       True       2DPt          2DPt
              North (  N)               South (  S)    False      False       True       2DPt          2DPt
          Southwest ( SW)           Northeast ( NE)    False      False       True       2DPt          2DPt
               West (  W)                East (  E)    False      False       True       2DPt          2DPt
          Northwest ( NW)           Southeast ( SE)    False      False       True       2DPt          2DPt
          Southeast ( SE)           Northwest ( NW)    False      False       True       2DPt          2DPt
           

In [4]:
qr.print_point_algebra_composition_table(ptalg2d)

2D_Point_Algebra
Elements: S, EQ, N, SW, W, NW, SE, E, NE
 rel1 ; rel2 = composition
   S      S      S
   S     EQ      S
   S      N      S|EQ|N
   S     SW      SW
   S      W      SW
   S     NW      SW|W|NW
   S     SE      SE
   S      E      SE
   S     NE      SE|E|NE
------------------------------
  EQ      S      S
  EQ     EQ      EQ
  EQ      N      N
  EQ     SW      SW
  EQ      W      W
  EQ     NW      NW
  EQ     SE      SE
  EQ      E      E
  EQ     NE      NE
------------------------------
   N      S      S|EQ|N
   N     EQ      N
   N      N      N
   N     SW      SW|W|NW
   N      W      NW
   N     NW      NW
   N     SE      SE|E|NE
   N      E      NE
   N     NE      NE
------------------------------
  SW      S      SW
  SW     EQ      SW
  SW      N      SW|W|NW
  SW     SW      SW
  SW      W      SW
  SW     NW      SW|W|NW
  SW     SE      S|SW|SE
  SW      E      S|SW|SE
  SW     NE      S|EQ|N|SW|W|NW|SE|E|NE
------------------------------
   W      S

## Derive Rectangle Relations

### Define a Four Point Network (Class) for 2D Points

In [5]:
class Four2DPointNet(qr.Network):
    """Create four 2D Points that represent the coordinates of the upper-right (UR)
    and lower-left (LL) of two rectangles. For example, (LL1, UR1) and (LL2, UR2),
    where LL1 SW UR1, LL2 SW UR2, and SW is the 2DPoint relation southwest,
    represents two proper rectangles. Return the network and the four spatial entities."""
    def __init__(self, algebra, name, southwest="SW", lowerleft="LL", upperright="UR"):
        self.algebra = algebra
        self.SW = algebra.relset(southwest)
        # LL and UR coordinates of rectangle 1
        LL1 = lowerleft + "1"
        UR1 = upperright + "1"
        self.LL1 = qr.SpatialEntity(["2DPoint"], name=LL1)
        self.UR1 = qr.SpatialEntity(["2DPoint"], name=UR1)
        # LL and UR coordinates of rectangle 2
        LL2 = lowerleft + "2"
        UR2 = upperright + "2"
        self.LL2 = qr.SpatialEntity(["2DPoint"], name=LL2)
        self.UR2 = qr.SpatialEntity(["2DPoint"], name=UR2)
        self.name_list = [LL1, UR1, LL2, UR2]
        super().__init__(algebra, name)
        self.add_constraint(self.LL1, self.UR1, self.SW, verbose=False)
        self.add_constraint(self.LL2, self.UR2, self.SW, verbose=False)

    def get_points(self):
        return [self.LL1, self.UR1, self.LL2, self.UR2]

In [6]:
four_pt_net = Four2DPointNet(ptalg2d, "FourPtNet")

four_pt_net

The following few cells are just checks to make sure the the four point network looks OK.

In [7]:
four_pt_net.get_points()

[SpatialEntity(['2DPoint'] 'LL1'),
 SpatialEntity(['2DPoint'] 'UR1'),
 SpatialEntity(['2DPoint'] 'LL2'),
 SpatialEntity(['2DPoint'] 'UR2')]

In [8]:
four_pt_net.summary()


FourPtNet: 4 nodes, 8 edges
  Algebra: 2D_Point_Algebra
  LL1:['2DPoint']
    => LL1: EQ
    => UR1: SW
  UR1:['2DPoint']
    => UR1: EQ
  LL2:['2DPoint']
    => LL2: EQ
    => UR2: SW
  UR2:['2DPoint']
    => UR2: EQ


In [9]:
four_pt_net.propagate()

True

In [10]:
four_pt_net.summary()


FourPtNet: 4 nodes, 16 edges
  Algebra: 2D_Point_Algebra
  LL1:['2DPoint']
    => LL1: EQ
    => UR1: SW
    => LL2: S|EQ|N|SW|W|NW|SE|E|NE
    => UR2: S|EQ|N|SW|W|NW|SE|E|NE
  UR1:['2DPoint']
    => UR1: EQ
    => LL2: S|EQ|N|SW|W|NW|SE|E|NE
    => UR2: S|EQ|N|SW|W|NW|SE|E|NE
  LL2:['2DPoint']
    => LL2: EQ
    => UR2: SW
  UR2:['2DPoint']
    => UR2: EQ


## Derive the Consistent Networks for Proper Rectangles

In [11]:
def generate_consistent_2D_networks(point_algebra, lessthan=["SW"], startname="LL", endname="UR",
                                    verbose=False):
    """For a given point algebra and southwest relation, derive all possible consistent networks
    of 4 points, where the points represent the lower-left and upper-right of 2 rectangles."""
    consistent_nets = dict()
    for elem13 in point_algebra.elements:
        for elem23 in point_algebra.elements:
            for elem14 in point_algebra.elements:
                for elem24 in point_algebra.elements:
                    four_pt_net_name = elem13 + ',' + elem14 + ',' + elem23 + ',' + elem24
                    ptnet = Four2DPointNet(point_algebra, four_pt_net_name, lessthan, startname, endname)
                    pt1, pt2, pt3, pt4 = ptnet.get_points()
                    rs13 = point_algebra.relset(elem13)
                    rs14 = point_algebra.relset(elem14)
                    rs23 = point_algebra.relset(elem23)
                    rs24 = point_algebra.relset(elem24)
                    ptnet.add_constraint(pt1, pt3, rs13)
                    ptnet.add_constraint(pt1, pt4, rs14)
                    ptnet.add_constraint(pt2, pt3, rs23)
                    ptnet.add_constraint(pt2, pt4, rs24)
                    if ptnet.propagate():
                        elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                        consistent_nets[elem_key] = ptnet
                        if verbose:
                            print(np.array(ptnet.to_list()))
    print(f"\n{len(consistent_nets)} consistent networks")
    return consistent_nets

When we run the code below (with lessthan=["SW"]) then we will get 169 (i.e., 13x13) consistent 4 point networks, <b>and we do!</b>

In that case, each network corresponds to a unique configuration of two <b>proper</b> rectangles.

By <b>proper rectangle</b> we mean that the rectangles cannot degenerate into segments or points.

If we set lessthan=["W", "SW"], then line segments (0 height) can be considered to be rectangles, and we will get 234 configurations.  If we use "S", instead of "W", then 0-width line segments can be considered to be rectangles, and we will again get 234 configurations.

If lessthan=["EQ", "W", "SW"], then points can be considered to be (degenerate) rectangles, and we will get 261 configurations.

lessthan=["EQ", "W", "S", "SW"] yields 324 configurations of two (possibly degenerate) rectangles.

In [12]:
result_proper = generate_consistent_2D_networks(ptalg2d, lessthan=["SW"], verbose=True)

[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'SE' 'S']
 ['N' 'NW' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'SE' 'SW']
 ['N' 'NW' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'SE' 'SE']
 ['N' 'NW' 'EQ' 'SW']
 ['NE' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'E' 'S']
 ['N' 'W' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'E' 'SW']
 ['N' 'W' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'E' 'SE']
 ['N' 'W' 'EQ' 'SW']
 ['NE' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'S']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'N']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'S' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'SW']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'W']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'E' 'NE' 'EQ']]
[['EQ' 'SW' 'S'

[['EQ' 'SW' 'SE' 'S']
 ['NE' 'EQ' 'SE' 'SE']
 ['NW' 'NW' 'EQ' 'SW']
 ['N' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SW']
 ['NE' 'EQ' 'SE' 'S']
 ['NW' 'NW' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SW']
 ['NE' 'EQ' 'SE' 'SW']
 ['NW' 'NW' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SW']
 ['NE' 'EQ' 'SE' 'SE']
 ['NW' 'NW' 'EQ' 'SW']
 ['NE' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SE']
 ['NE' 'EQ' 'SE' 'SE']
 ['NW' 'NW' 'EQ' 'SW']
 ['NW' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'S']
 ['NE' 'EQ' 'E' 'SE']
 ['NW' 'W' 'EQ' 'SW']
 ['N' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SW']
 ['NE' 'EQ' 'E' 'S']
 ['NW' 'W' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SW']
 ['NE' 'EQ' 'E' 'SW']
 ['NW' 'W' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SW']
 ['NE' 'EQ' 'E' 'SE']
 ['NW' 'W' 'EQ' 'SW']
 ['NE' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'SE']
 ['NE' 'EQ' 'E' 'SE']
 ['NW' 'W' 'EQ' 'SW']
 ['NW' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'SE' 'S']
 ['NE' 'EQ' 'NE' 'SE']
 ['NW' 'SW' 'EQ' 'SW']
 ['N' 'NW' 'NE' 'EQ'

In [13]:
len(result_proper)

169

In [14]:
result_all = generate_consistent_2D_networks(ptalg2d, lessthan=["EQ", "W", "S", "SW"], verbose=True)

[['EQ' 'S|EQ' 'S' 'S']
 ['EQ|N' 'EQ' 'S' 'S']
 ['N' 'N' 'EQ' 'S|EQ']
 ['N' 'N' 'EQ|N' 'EQ']]
[['EQ' 'S|EQ' 'S' 'SW']
 ['EQ|N' 'EQ' 'S' 'SW']
 ['N' 'N' 'EQ' 'SW|W']
 ['NE' 'NE' 'E|NE' 'EQ']]
[['EQ' 'S' 'S' 'S']
 ['N' 'EQ' 'EQ' 'S']
 ['N' 'EQ' 'EQ' 'S']
 ['N' 'N' 'N' 'EQ']]
[['EQ' 'S' 'S' 'S']
 ['N' 'EQ' 'EQ' 'EQ']
 ['N' 'EQ' 'EQ' 'EQ']
 ['N' 'EQ' 'EQ' 'EQ']]
[['EQ' 'S' 'S' 'SW']
 ['N' 'EQ' 'EQ' 'SW']
 ['N' 'EQ' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'S' 'S' 'SW']
 ['N' 'EQ' 'EQ' 'W']
 ['N' 'EQ' 'EQ' 'W']
 ['NE' 'E' 'E' 'EQ']]
[['EQ' 'S' 'S' 'S']
 ['N' 'EQ' 'N' 'S']
 ['N' 'S' 'EQ' 'S']
 ['N' 'N' 'N' 'EQ']]
[['EQ' 'S' 'S' 'S']
 ['N' 'EQ' 'N' 'EQ']
 ['N' 'S' 'EQ' 'S']
 ['N' 'EQ' 'N' 'EQ']]
[['EQ' 'S' 'S' 'S']
 ['N' 'EQ' 'N' 'N']
 ['N' 'S' 'EQ' 'S|EQ']
 ['N' 'S' 'EQ|N' 'EQ']]
[['EQ' 'S' 'S' 'SW']
 ['N' 'EQ' 'N' 'SW']
 ['N' 'S' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'S' 'S' 'SW']
 ['N' 'EQ' 'N' 'W']
 ['N' 'S' 'EQ' 'SW']
 ['NE' 'E' 'NE' 'EQ']]
[['EQ' 'S' 'S' 'SW']
 ['N' 'EQ' 'N' 'NW']
 

[['EQ' 'SW|W' 'SW' 'SW']
 ['E|NE' 'EQ' 'S' 'SW']
 ['NE' 'N' 'EQ' 'SW|W']
 ['NE' 'NE' 'E|NE' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'EQ' 'S']
 ['NE' 'EQ' 'EQ' 'S']
 ['NE' 'N' 'N' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'EQ' 'EQ']
 ['NE' 'EQ' 'EQ' 'EQ']
 ['NE' 'EQ' 'EQ' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'EQ' 'SW']
 ['NE' 'EQ' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'EQ' 'W']
 ['NE' 'EQ' 'EQ' 'W']
 ['NE' 'E' 'E' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'N' 'S']
 ['NE' 'S' 'EQ' 'S']
 ['NE' 'N' 'N' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'N' 'EQ']
 ['NE' 'S' 'EQ' 'S']
 ['NE' 'EQ' 'N' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'N' 'N']
 ['NE' 'S' 'EQ' 'S|EQ']
 ['NE' 'S' 'EQ|N' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'N' 'SW']
 ['NE' 'S' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'N' 'W']
 ['NE' 'S' 'EQ' 'SW']
 ['NE' 'E' 'NE' 'EQ']]
[['EQ' 'SW' 'SW' 'SW']
 ['NE' 'EQ' 'N' 'NW']
 ['NE' 'S' 'EQ' 'SW|W']
 ['NE' 'SE' 'E|NE' 

[['EQ' 'SW|W' 'NW' 'SW']
 ['E|NE' 'EQ' 'NE' 'S']
 ['SE' 'SW' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'NW' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']
 ['SE' 'SW' 'EQ' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']]
[['EQ' 'SW' 'NW' 'SW']
 ['NE' 'EQ' 'NE' 'N']
 ['SE' 'SW' 'EQ' 'SW']
 ['NE' 'S' 'NE' 'EQ']]
[['EQ' 'SW|W' 'NW' 'SW']
 ['E|NE' 'EQ' 'NE' 'SW']
 ['SE' 'SW' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'NW' 'SW']
 ['NE' 'EQ' 'NE' 'W']
 ['SE' 'SW' 'EQ' 'SW']
 ['NE' 'E' 'NE' 'EQ']]
[['EQ' 'SW' 'NW' 'SW']
 ['NE' 'EQ' 'NE' 'NW']
 ['SE' 'SW' 'EQ' 'SW']
 ['NE' 'SE' 'NE' 'EQ']]
[['EQ' 'SW|W' 'NW' 'SW']
 ['E|NE' 'EQ' 'NE' 'SE']
 ['SE' 'SW' 'EQ' 'S|SW']
 ['NE' 'NW' 'N|NE' 'EQ']]
[['EQ' 'SW' 'NW' 'SW']
 ['NE' 'EQ' 'NE' 'E']
 ['SE' 'SW' 'EQ' 'S|SW']
 ['NE' 'W' 'N|NE' 'EQ']]
[['EQ' 'SW' 'NW' 'SW']
 ['NE' 'EQ' 'NE' 'NE']
 ['SE' 'SW' 'EQ' 'S|SW']
 ['NE' 'SW' 'N|NE' 'EQ']]
[['EQ' 'W' 'NW' 'W']
 ['E' 'EQ' 'NE' 'EQ']
 ['SE' 'SW' 'EQ' 'SW']
 ['E' 'EQ' 'NE' 'EQ']]
[['EQ' 'SW' 'NW' 'W']
 ['NE' 'EQ' 'NE' 'N']
 ['SE' 'SW' 'E

[['EQ' 'S|EQ' 'NE' 'S']
 ['EQ|N' 'EQ' 'NE' 'S']
 ['SW' 'SW' 'EQ' 'SW']
 ['N' 'N' 'NE' 'EQ']]
[['EQ' 'S' 'NE' 'S']
 ['N' 'EQ' 'NE' 'EQ']
 ['SW' 'SW' 'EQ' 'SW']
 ['N' 'EQ' 'NE' 'EQ']]
[['EQ' 'S' 'NE' 'S']
 ['N' 'EQ' 'NE' 'N']
 ['SW' 'SW' 'EQ' 'SW']
 ['N' 'S' 'NE' 'EQ']]
[['EQ' 'SW|W' 'NE' 'S']
 ['E|NE' 'EQ' 'NE' 'SE']
 ['SW' 'SW' 'EQ' 'SW']
 ['N' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'NE' 'S']
 ['NE' 'EQ' 'NE' 'E']
 ['SW' 'SW' 'EQ' 'SW']
 ['N' 'W' 'NE' 'EQ']]
[['EQ' 'SW' 'NE' 'S']
 ['NE' 'EQ' 'NE' 'NE']
 ['SW' 'SW' 'EQ' 'SW']
 ['N' 'SW' 'NE' 'EQ']]
[['EQ' 'EQ' 'NE' 'EQ']
 ['EQ' 'EQ' 'NE' 'EQ']
 ['SW' 'SW' 'EQ' 'SW']
 ['EQ' 'EQ' 'NE' 'EQ']]
[['EQ' 'S' 'NE' 'EQ']
 ['N' 'EQ' 'NE' 'N']
 ['SW' 'SW' 'EQ' 'SW']
 ['EQ' 'S' 'NE' 'EQ']]
[['EQ' 'W' 'NE' 'EQ']
 ['E' 'EQ' 'NE' 'E']
 ['SW' 'SW' 'EQ' 'SW']
 ['EQ' 'W' 'NE' 'EQ']]
[['EQ' 'SW' 'NE' 'EQ']
 ['NE' 'EQ' 'NE' 'NE']
 ['SW' 'SW' 'EQ' 'SW']
 ['EQ' 'SW' 'NE' 'EQ']]
[['EQ' 'S|EQ' 'NE' 'N']
 ['EQ|N' 'EQ' 'NE' 'N']
 ['SW' 'SW' 'EQ' 'SW|W']
 ['S' 'S' 'E|NE' 'E

In [15]:
len(result_all)

324

## Assigning Names to the Rectangle Relations

WORK-IN-PROGRESS (watch this space...)

In [16]:
def relative_position(pt_rel):
    vert_pos = None
    horiz_pos = None

    if pt_rel in ["NW",  "N", "NE"]:
        vert_pos = "above"
    elif pt_rel in ["SW",  "S", "SE"]:
        vert_pos = "below"
    else:
        vert_pos = "vert_eq"
        
    if pt_rel in ["E", "NE", "SE"]:
        horiz_pos = "right"
    elif pt_rel in ["W", "NW", "SW"]:
        horiz_pos = "left"
    else:
        horiz_pos = "horiz_eq"
    
    return horiz_pos, vert_pos

In [17]:
for rel in ptalg2d.elements:
    print(rel, relative_position(rel))

S ('horiz_eq', 'below')
EQ ('horiz_eq', 'vert_eq')
N ('horiz_eq', 'above')
SW ('left', 'below')
W ('left', 'vert_eq')
NW ('left', 'above')
SE ('right', 'below')
E ('right', 'vert_eq')
NE ('right', 'above')


...MORE TO COME...